## Importation librairie et donnée


In [5]:
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
import re
import os
import time
import matplotlib.pyplot as plt
from google.colab import files
from google.colab import drive

In [6]:
drive.mount('/content/drive')
# Chemin du dossier dans votre Google Drive
chemin_dossier_pln_janvier = '/content/drive/My Drive/data/stan_janvier_2024'
chemin_dossier_pln_mars = '/content/drive/My Drive/data/stan_mars_2024'
chemin_dossier_abacus = '/content/drive/My Drive/data/ABACUS'
chemin_dossier_table_systeme = '/content/drive/My Drive/data/TABLE_SYSTEME'
chemin_dossier_table_fusion = '/content/drive/My Drive/data/fusion_radar'

# Liste des fichiers dans le dossier
contenu_dossier_pln_janvier = os.listdir(chemin_dossier_pln_janvier)
contenu_dossier_pln_mars = os.listdir(chemin_dossier_pln_mars)
contenu_dossier_abacus = os.listdir(chemin_dossier_abacus)
contenu_dossier_table_systeme = os.listdir(chemin_dossier_table_systeme)
contenu_dossier_table_fusion = os.listdir(chemin_dossier_table_fusion)

Mounted at /content/drive


In [2]:
pd.set_option('display.max_columns', 150)

In [15]:
def read_and_process_file(fichier_a_deposee):
    vol_prevu = []
    vol_fini = []
    vol_termine = []
    tableau_vol = {}
    tableaux_vol = []
    iter = 0
    flag82 = False
    hneg = False
    compt82 = 0
    num81 = 0
    prevu = False
    termine = False
    final = False
    complet = 0
    output = pd.DataFrame()
    compteur = 0
    tableau_vol = {}
    compteurCcr = 0

    with open(fichier_a_deposee, 'r') as fichier:
        for i, ligne in enumerate(fichier):
            words = ligne.split()
            if words[0] == "02":
                date_str = words[1]
                date_obj = datetime.strptime(date_str, "%d-%m-%Y")
                date_fichier = date_obj.timetuple().tm_yday
            if words[0] == "05":
                if tableau_vol:
                    tableau_vol["isPrevu"] = isprevu
                    tableau_vol["isRealise"] = isrealise
                    tableau_vol["isFinal"] = isfinal
                    df_dictionary = pd.DataFrame([tableau_vol])
                    output = pd.concat([output, df_dictionary], ignore_index=True)
                tableau_vol = {}
                isprevu = False
                isrealise = False
                isfinal = False
                is81 = False
                is82 = False
                balise = ''
                hneg = False
                flag82 = False
                etat = ''
            if words[0] == "11":
                etat = 'prevu'
                isprevu = True
            if words[0] == "12":
                etat = 'realise'
                isrealise = True
            if words[0] == "13":
                if len(words) > 1 and words[1] == "=":
                    for key in list(tableau_vol.keys()):
                        if '_prevu' in key:
                            tableau_vol[key.replace('_prevu', '_final')] = tableau_vol[key]
                etat = 'final'
                isfinal = True
            if words[0] == "14":
                etat = 'transaction'
            if words[0] == "81":
                pass
            if words[0] == "20":
                tableau_vol['callSign_' + etat] = words[1]
                tableau_vol['dep_' + etat] = words[2]
                tableau_vol['arr_' + etat] = words[3]
                tableau_vol['numCautra_' + etat] = words[4]
                tableau_vol['dateRelative_' + etat] = words[5]
                tableau_vol['typeAvion_' + etat] = words[6]
                tableau_vol['work_' + etat] = words[7]
                if words[8][:2] == '??':
                    pass
                else:
                    tableau_vol['work1' + etat] = words[8].strip().ljust(9)
            if words[0] == "21":
                tableau_vol['heuresDep_' + etat] = words[1]
                tableau_vol['RFL_' + etat] = words[2]
                tableau_vol['vitesse_' + etat] = words[3]
                tableau_vol['EOBT_' + etat] = words[4]
            if words[0] == "22":
                tableau_vol['regleVol_' + etat] = words[1]
                tableau_vol['typeVol_' + etat] = words[2]
                tableau_vol['HeurePremiereBaliseActive_' + etat] = words[10]
                if words[3][:2] == '??':
                    pass
                else:
                    tableau_vol['IFPL_' + etat] = words[3].strip().ljust(10)
                tableau_vol['plnActive_' + etat] = words[4]
                tableau_vol['typePlnStan']= words[6]
                tableau_vol['plnAnnule_' + etat] = words[5]
                if '??' in words[7]:
                    pass
                elif len(words[7]) == 8:
                    date_str = words[7]
                    date_obj = datetime.strptime(date_str, '%d%m%Y')
                    day_vol = date_obj.timetuple().tm_yday
                    tableau_vol['dateBlock_' + etat] = words[7][:4] + words[7][6:]
                else:
                    tableau_vol['dateBlock_' + etat] = words[7].strip().ljust(6)
            if words[0] == "23":
                if "??" in words[4]:
                    tableau_vol['adresseModeS_' + etat] = np.NaN
                else:
                    tableau_vol['adresseModeS_' + etat] = words[4]
            if words[0] == "24":
                tableau_vol['numeroPLNM' + etat] = words[1]
                tableau_vol['flightID' + etat] = words[2]
            if words[0] == "31":
                tableau_vol['balise' + etat] = words[1]
            if words[0] == "32":
                tableau_vol['HeurePremiereBalise_' + etat] = words[1]
            if words[0] == "33":
                tableau_vol['listeBalises' + etat] = words[1]
            if words[0] == "36":
                tableau_vol['indicateur' + etat] = words[1]
            if words[0] == "41":
                tableau_vol['carte' + etat] = words[1]
            if words[0] == "71":
                tableau_vol['centreTraversé' + etat] = words[1:]
            if words[0] == "72":
                tableau_vol['listeRangPremier' + etat] = words[1]
            if words[0] == "80":
                tableau_vol['rangTransaction' + etat] = words[1]
            if words[0] == "81" and not is81:
                is81 = True
                if len(words) >= 15:
                    parts = ligne.split("-")
                    last_word = parts[0].split()[-1]
                    if "ABI" in ligne:
                        tableau_vol['typePln'] = "ABI"
                    if "(FPL" in parts[0] or "(CHG)" in parts[0]:
                        tableau_vol['case7'] = parts[1]
                        tableau_vol['case8'] = parts[2]
                        tableau_vol['case9'] = parts[3]
                        tableau_vol['case10'] = parts[4]
                        tableau_vol['case13'] = parts[5]
                        tableau_vol['case15'] = parts[6]
                        if len(parts) > 8:
                            tableau_vol['case16'] = parts[7]
                            tableau_vol['case18'] = parts[8]
                            if tableau_vol['case18'] == "RPL":
                                tableau_vol['typePln'] = "RPL"
                        else:
                            #print(ligne)
                            compteur += 1
                    elif "(APL" in parts[0]:
                        tableau_vol['case7'] = parts[1]
                        tableau_vol['case8'] = parts[2]
                        tableau_vol['case9'] = parts[3]
                        tableau_vol['case10'] = parts[4]
                        tableau_vol['case13'] = parts[5]
                        tableau_vol['case15'] = parts[6]
                        tableau_vol['typePln'] = "APL"
                        if len(parts) > 8:
                            tableau_vol['case16'] = parts[7]
                            tableau_vol['case18'] = parts[-1]
                        else:
                            compteur += 1
            if words[0] == "82"and not is82:
                is82 = True
                tableau_vol['heure'] = (words[1][:2])
                tableau_vol['minute'] = (words[1][3:])
                tableau_vol['accuseTrt' + etat] = words[1]
                if "CCR:" in ligne:
                    compteurCcr = 0
                    for word in words:
                        compteurCcr += 1
                        if word == "CCR:":
                            break
                    tableau_vol['ccrArrival'] = words[compteurCcr]
            if words[0] == "84":
                tableau_vol['final' + etat] = words[1]

    return output

def convert_and_calculate(df,date_obj):
    df['HeurePremiereBaliseActive_realise'] = df['HeurePremiereBaliseActive_realise'].astype('Int64')
    df['HeurePremiereBaliseActive_final'] = df['HeurePremiereBaliseActive_final'].astype('Int64')
    df['HeurePremiereBalise_final'] = df['HeurePremiereBalise_final'].astype('Int64')
    df['dateRelative_realise'] = df['dateRelative_realise'].astype('Int64')
    df['dateRelative_final'] = df['dateRelative_final'].astype('Int64')

    def calcul_HeureDeReference(row):
        try:
            if not pd.isna(row['dateRelative_realise']) and not pd.isnull(row['dateRelative_realise']):
                if not pd.isna(row['HeurePremiereBaliseActive_realise']) and int(row['HeurePremiereBaliseActive_realise']) != 0:
                    return int(row['HeurePremiereBaliseActive_realise']) + (-1440 if row['dateRelative_realise'] == -1 else 1440 if row['dateRelative_realise'] == 1 else 0)
                elif not pd.isna(row['HeurePremiereBalise_final']):
                    return int(row['HeurePremiereBalise_final']) + (-1440 if row['dateRelative_realise'] == -1 else 1440 if row['dateRelative_realise'] == 1 else 0)
            elif not pd.isna(row['dateRelative_final']):
                if not pd.isna(row['HeurePremiereBaliseActive_final']) and int(row['HeurePremiereBaliseActive_final']) != 0:
                    return int(row['HeurePremiereBaliseActive_final']) + (-1440 if row['dateRelative_final'] == -1 else 1440 if row['dateRelative_final'] == 1 else 0)
                elif not pd.isna(row['HeurePremiereBalise_final']):
                    return int(row['HeurePremiereBalise_final']) + (-1440 if row['dateRelative_final'] == -1 else 1440 if row['dateRelative_final'] == 1 else 0)
        except Exception:
            return None

    df['heure_de_reference'] = df.apply(calcul_HeureDeReference, axis=1)

    def calcul_DateDeReference(row):
        try:
            if not pd.isna(row['dateRelative_realise']) and not pd.isnull(row['dateRelative_realise']):
                if row['dateRelative_realise'] == 0:
                    return date_obj
                elif row['dateRelative_realise'] == 1:
                    return date_obj + timedelta(days=1)
                elif row['dateRelative_realise'] == -1 and int(row['heure_de_reference'])<0:
                    return date_obj - timedelta(days=1)
                else:
                    return date_obj
            elif not pd.isna(row['dateRelative_final']) and not pd.isnull(row['dateRelative_final']):
                if row['dateRelative_final'] == 0:
                    return date_obj
                elif row['dateRelative_final'] == 1:
                    return date_obj + timedelta(days=1)
                elif row['dateRelative_final'] == -1 and int(row['heure_de_reference'])<0:
                    return date_obj - timedelta(days=1)
                else:
                    return date_obj
        except Exception as e:
            return None

    df['date_de_reference'] = df.apply(calcul_DateDeReference, axis=1)
    return df

In [17]:
from datetime import datetime, timedelta

def dynamic_analysis(target_date_str):
    # Convert the target_date_str to a datetime object
    target_date = datetime.strptime(target_date_str, "%d-%m-%Y")

    # Calculate the previous and next days
    previous_day = target_date - timedelta(days=1)

    next_day = target_date + timedelta(days=1)


    # Format the dates to match the file naming convention
    previous_day_str = previous_day.strftime("%Y%m%d")
    target_date_str = target_date.strftime("%Y%m%d")
    next_day_str = next_day.strftime("%Y%m%d")

    # Construct file names
    nom_fichier_jour_1 = f'/content/drive/My Drive/data/stan_mars_2024/RDVC-{previous_day_str}.pln'
    nom_fichier_jour_2 = f'/content/drive/My Drive/data/stan_mars_2024/RDVC-{target_date_str}.pln'
    nom_fichier_jour_3 = f'/content/drive/My Drive/data/stan_mars_2024/RDVC-{next_day_str}.pln'

    # Read and process files
    fichier_jour_1 = read_and_process_file(nom_fichier_jour_1)
    fichier_jour_2 = read_and_process_file(nom_fichier_jour_2)
    fichier_jour_3 = read_and_process_file(nom_fichier_jour_3)

    # Convert and calculate
    date_obj = previous_day
    jour_1 = convert_and_calculate(fichier_jour_1,date_obj)

    date_obj = target_date
    jour_2 = convert_and_calculate(fichier_jour_2,date_obj)

    date_obj = next_day
    jour_3 = convert_and_calculate(fichier_jour_3,date_obj)


    # Return the results
    return jour_1, jour_2, jour_3

# Example usage
dateAnalyse='21-03-2024'
jour_1, jour_2, jour_3 = dynamic_analysis(dateAnalyse)

# Algo 2.3 et 2.4

## Tables utilisées

In [18]:
INDICATIF_FICTIF = pd.read_csv(os.path.join(chemin_dossier_table_systeme,"indicatifsfictifs"), sep=';', skiprows=3)
EUROPE_SUD = pd.read_csv(os.path.join(chemin_dossier_table_systeme,"payssud"), sep=';', skiprows=3)
AERONEFS_DE_MOINS_DE_2_TONNES = pd.read_csv(os.path.join(chemin_dossier_table_systeme,"aeronefsmoin2tonnes"), sep=';', skiprows=3)
INDICATEURS_D_EMPLACEMENT_FAUX = pd.read_csv(os.path.join(chemin_dossier_table_systeme,"aerodromesfaux"), sep=';', skiprows=3)
AERODROME_A_VERIFIER = pd.read_csv(os.path.join(chemin_dossier_table_systeme,"aerodromesaverifier"), sep=';', skiprows=3)
AERODROMES_D_APPROCHE = pd.read_csv(os.path.join(chemin_dossier_table_systeme,"aerodromesapproche"), sep=';', skiprows=3)
AERODROMES_FRONTALIERS = pd.read_csv(os.path.join(chemin_dossier_table_systeme,"aerodromesfrontaliers"), sep=';', skiprows=3)
INDICATIF_A_CODE_AUTO = pd.read_csv(os.path.join(chemin_dossier_table_systeme,"indicatifscodesauto"), sep=';', skiprows=3)
INDICATIFS_A_STRUCTURE_TRIGRAMME_CORRECTE = pd.read_csv(os.path.join(chemin_dossier_table_systeme,"structurtrigrammecorrects"), sep=';', skiprows=3)
COMPAGNIES_BIGRAMMES_ET_SUFFIXES = pd.read_csv(os.path.join(chemin_dossier_table_systeme,"cgniebigrammesetsuffixes"), sep=';', skiprows=3)
COMPAGNIES_AVEC_TRIGRAMME = pd.read_csv(os.path.join(chemin_dossier_table_systeme,"compagniesavectrigrammesautorise"), sep=';', skiprows=3)
INDICATIFS_TRIGRAMME_A_VERIFIER = pd.read_csv(os.path.join(chemin_dossier_table_systeme,"structuretrigrammeverif"), sep=';', skiprows=3)
LETTRES_AIR_FRANCE = pd.read_csv(os.path.join(chemin_dossier_table_systeme,"lettresairfrance"), sep=';', skiprows=3)
LETTRES_AIR_FRANCE = LETTRES_AIR_FRANCE[~((LETTRES_AIR_FRANCE['Lettre'].str.contains("MODIFIE", na=False)) & (LETTRES_AIR_FRANCE['Code exo/trait'].isna()) & (LETTRES_AIR_FRANCE['Commentaire'].isna()))]
LETTRES_AIR_FRANCE.reset_index(drop=True, inplace=True)
IMMATRICULATION_A_VERIFIER = pd.read_csv(os.path.join(chemin_dossier_table_systeme,"immataverif"), sep=';', skiprows=3)
IMMATRICULATION_CORRECTE = pd.read_csv(os.path.join(chemin_dossier_table_systeme,"immatcorrectes"), sep=';', skiprows=3)
OPERATEURS_MILITAIRES = pd.read_csv(os.path.join(chemin_dossier_table_systeme,"oprmili"), sep=';', skiprows=3)
AERONEFS_STRICTEMENT_MILITAIRES = pd.read_csv(os.path.join(chemin_dossier_table_systeme,"aeronefmilitaire"), sep=';', skiprows=3)

In [19]:
# Fonction pour convertir un modèle en expression régulière
def pattern_to_regex(pattern):
    # Remplacer # par .*, + par [a-zA-Z] et - par [0-9]
    regex = pattern.replace('#', '.?').replace('-', '[0-9]').replace('+', '[a-zA-Z]')
    # Ajouter les délimiteurs de début et de fin
    regex = '^' + regex + '$'
    return regex

def trouver_pattern(x, pattern_list):
    for pattern in pattern_list:
        regex = pattern_to_regex(pattern)
        if re.match(regex, x):
            return True
    return False

def renvoie_pattern(x, pattern_list):
    for pattern in pattern_list:
        regex = pattern_to_regex(pattern)
        if re.match(regex, x):
            return pattern
    return "NULL"

## Prétraitement utile/inutile

In [20]:
def nbre_centre_francais(list):
    rep = 0
    for x in list:
      if x in ['PARI', 'AIX', 'BRST', 'REIM', 'BORD']:
        rep +=1
    return rep

In [21]:
def utile_inutile(element):
    # Fonction pour trouver une valeur valide dans les colonnes multiples
    def get_valid_value(element, columns):
        for col in columns:
            if col in element and not(element[[col]].isna().iloc[0]):
                return element[col]
        return None

    call_sign_value = get_valid_value(element, ["callSign_realise", "callSign_final", "callSign_prevu"])
    PLN_active_value = get_valid_value(element, ["plnActive_realise", "plnActive_final", "plnActive_prevu"])
    PLN_annule_value = get_valid_value(element, ["plnAnnule_realise", "plnAnnule_final", "plnAnnule_prevu"])
    centre_traverse_value = get_valid_value(element, ["centreTraversérealise", "centreTraverséfinal", "centreTraverséprevu"])
    dep_value = get_valid_value(element, ["dep_realise", "dep_final", "dep_prevu"])
    arr_value = get_valid_value(element, ["arr_realise", "arr_final", "arr_prevu"])

    if trouver_pattern(call_sign_value, INDICATIF_FICTIF["Indicatif"].to_list()):
        return "FICT"
    if PLN_active_value == "1":
        return "UTI"
    elif PLN_annule_value == "1":
        return "CNL"
    elif nbre_centre_francais(centre_traverse_value) >= 2:
        return "2SLF"
    elif dep_value == arr_value:
        return "CIRC"
    elif dep_value[0] == "E" and dep_value in EUROPE_SUD["Code pays"].to_list():
        return "NORD"
    elif (dep_value[0] in EUROPE_SUD["Code pays"].to_list() or dep_value[:2] in EUROPE_SUD) and dep_value[0] == "E":
        return "SUD"
    return "UTI"


In [22]:
def traitement_utile_inutile(df):
    df['utile_inutile'] = df.apply(lambda x: utile_inutile(x), axis=1)
    return df

In [23]:
def test_utile_vol_active_non_fictif():
    utilite = result[result["callSign_prevu"]=="TRA79Y"]["utile_inutile"].iloc[0]
    assert utilite == "UTI", "utilité diff de UTI"

def test_inutile_vol_fictif_avec_moins():
    utilite = result[result["callSign_prevu"]=="RFIN65"]["utile_inutile"].iloc[0]
    assert utilite == "FICT", "utilité diff de FICT"

def test_inutile_vol_annule():
    utilite = result[result["callSign_prevu"]=="THY90F"]["utile_inutile"].iloc[0]
    assert utilite == "CNL", "utilité diff de CNL"

def test_inutile_vol_centrefr():
    utilite = result[result["callSign_prevu"]=="TRA4N"]["utile_inutile"].iloc[0]
    assert utilite == "2SLF", "utilité diff de 2SLF"

def test_inutile_vol_circulaire():
    utilite = result[result["callSign_prevu"]=="FNY5118"]["utile_inutile"].iloc[0]
    assert utilite == "CIRC", "utilité diff de CIRC"

def test_utile_vol_tout_les_test_faux():
    utilite = result[result["callSign_prevu"]=="TOM22B"]["utile_inutile"].iloc[0]
    assert utilite == "UTI", "utilité diff de UTI"

def test_realise_pln_nan():
    utilite = result[result["callSign_prevu"]=="THY90F"]["utile_inutile"].iloc[0]
    assert utilite == "CNL", "utilité diff de CNL"

def test_realise_centre_nan():
    utilite = result[result["callSign_prevu"]=="UAL71"]["utile_inutile"].iloc[0]
    assert utilite == "2SLF", "utilité diff de 2SLF"

## Traitement unitaire

## Initialisation

In [24]:
def TU_init(df_utile):
    df_utile['aeronef_de_moins_de_deux_tonnes'] = False
    df_utile['vol_a_transmettre'] = False
    df_utile['vol_approche'] = False
    df_utile['vol_interieur'] = False
    df_utile['vol_frontalier'] = False
    df_utile['vol_VFR'] = False
    df_utile["code_d_exoneration"] = 'Z'
    df_utile['code_exploitant'] = 'Z'
    df_utile['compagnie_française'] = False
    df_utile['type_d_avion_militaire'] = False
    df_utile['PLN_à_verifier_TU'] = False
    df_utile['PLN_valide'] = True
    df_utile['RAZ_des_invalidites_de_type_TU'] = False
    df_utile['invalidite_TU'] = [[] for _ in range(len(df_utile))]
    df_utile['type_d_indicatif'] = ""
    return df_utile

## Algo 1

In [25]:
def TU_1(df_utile):
    def TU_1_element(x):
        # Vérification de PLN_active
        for col in ["plnActive_realise", "plnActive_final", "plnActive_prevu"]:
            if col in x and not pd.isna(x[col]):
                if x[col] == "0":
                    x['invalidite_TU'] = x.get('invalidite_TU', []) + ["NACT"]
                break

        # Vérification de typeavion
        for col in ["typeAvion_realise", "typeAvion_final", "typeAvion_prevu"]:
            if col in x and not pd.isna(x[col]):
                if x[col] == "ZZZZ":
                    x['invalidite_TU'] = x.get('invalidite_TU', []) + ["TYPAV"]
                    x['PLN_valide'] = False
                if x[col] in AERONEFS_DE_MOINS_DE_2_TONNES["Type avion"].to_list():
                    x['aeronef_de_moins_de_deux_tonnes'] = True
                break
        return x
    df_utile = df_utile.apply(TU_1_element, axis=1)
    return df_utile

## Algo 2

In [26]:
# def TU_2(df_utile):
#     def TU_2_element(x):
#         if len(x['depfinal']) != 4:
#             x['invalidite_TU'].append("DEPAR1")
#             x['PLN_valide'] = False
#         elif x['depfinal'] in INDICATEURS_D_EMPLACEMENT_FAUX:
#             x['invalidite_TU'].append("DEPAR2")
#             x['PLN_valide'] = False
#         elif trouver_pattern(x['depfinal'], AERODROME_A_VERIFIER):
#             x['invalidite_TU'].append("DVPAR3")
#             x['PLN_valide'] = False
#         if len(x['arrfinal']) != 4:
#             x['invalidite_TU'].append("ARRIV1")
#             x['PLN_valide'] = False
#         elif x['arrfinal'] in INDICATEURS_D_EMPLACEMENT_FAUX:
#             x['invalidite_TU'].append("ARRIV2")
#             x['PLN_valide'] = False
#         elif trouver_pattern(x['arrfinal'], AERODROME_A_VERIFIER):
#             x['invalidite_TU'].append("AVRIV3")
#             x['PLN_valide'] = False
#     df_utile = df_utile.apply(TU_2_element, axis=1)
#     return df_utile

In [27]:
def TU_2(df_utile):
    def TU_2_element(x):
        # Vérification des colonnes de départ
        for col in ["dep_realise", "dep_final", "dep_prevu"]:
            if col in x and not pd.isna(x[col]):
                if len(x[col]) != 4:
                    x['invalidite_TU'].append("DEPAR1")
                    x['PLN_valide'] = False
                elif trouver_pattern(x[col], INDICATEURS_D_EMPLACEMENT_FAUX["Code terrain"]):
                    x['invalidite_TU'].append("DEPAR2")
                    x['PLN_valide'] = False
                elif trouver_pattern(x[col], AERODROME_A_VERIFIER["Code terrain"]):
                    x['invalidite_TU'].append("DVPAR3")
                    x['PLN_valide'] = False
            break

        # Vérification des colonnes d'arrivée
        for col in ["arr_realise", "arr_final", "arr_prevu"]:
            if col in x and not pd.isna(x[col]):
                if len(x[col]) != 4:
                    x['invalidite_TU'].append("ARRIV1")
                    x['PLN_valide'] = False
                elif trouver_pattern(x[col], INDICATEURS_D_EMPLACEMENT_FAUX["Code terrain"]):
                    x['invalidite_TU'].append("ARRIV2")
                    x['PLN_valide'] = False
                elif trouver_pattern(x[col], AERODROME_A_VERIFIER["Code terrain"]):
                    x['invalidite_TU'].append("AVRIV3")
                    x['PLN_valide'] = False
            break
        return x
    df_utile = df_utile.apply(TU_2_element, axis=1)
    return df_utile

## Algo 3

In [28]:
def TU_3_bis(df_utile):
    eurocontrol = ['EB','ED','EF','EG','EH','EI','EK','EL','EN','EP','ES','EV','EY','LA','LB','LC','LD','LE','LG','LH','LI','LJ','LK','LM','LO','LP','LQ','LR','LS','LT','LU','LW','LY','LZ','UD','UG','GC']
    def TU_3_bis_element(x):
        dep_value = None
        for col in ["dep_realise", "dep_final", "dep_prevu"]:
            if col in x and not pd.isna(x[col]) and x[col] != "":
                dep_value = x[col]
                break
        if dep_value == "ZZZZ":
            x['PLN_valide'] = False
            x['PLN_à_verifier_TU'] = False
            x['invalidite_TU'].extend(["DEPAR2", "TRANS1"])
        elif dep_value[:2] in eurocontrol:
            x['vol_a_transmettre'] = False
        elif x["ccrArrival"] == "ALGR":
            x['vol_a_transmettre'] = True
        elif x["ccrArrival"] == "ETRG":
            if ((x['dep_prevu'][:1]=='D' or x['dep_prevu'][:1]=='F') and 'TABOT' in x['case15']):
                x['vol_a_transmettre'] = True
        else:
            x['vol_a_transmettre'] = False
        return x

    df_utile = df_utile.apply(TU_3_bis_element, axis=1)
    return df_utile

In [29]:
# def TU_3(df_utile):
#     def TU_3_element(x):
#         if x["depfinal"] == "ZZZZ":
#             x['PLN_valide'] = False
#             x['PLN_à_verifier_TU'] = False
#             x['invalidite_TU'].append("DEPAR2")
#             x['invalidite_TU'].append("TRANS1")
#         else:
#             if x["depfinal"][:2] in liste_origin_vol | x["depfinal"][:2] in liste_origin_vol:
#                 if trouver_code(x["depfinal"], liste_origin_vol) == "A":
#                     x['vol_a_transmettre'] = False
#                 elif trouver_code(x["depfinal"], liste_origin_vol) == "B":
#                     x['vol_a_transmettre'] = True
#                 elif trouver_code(x["depfinal"], liste_origin_vol) == "C":
#                     if x["AVR"] == 1:
#                         x['vol_a_transmettre'] = "ADET"
#                         x["PLN_valide"] = False
#                         x['invalidite_TU'].append("TRANSC")
#                     else:
#                         x['vol_a_transmettre'] = False
#                 elif trouver_code(x["depfinal"], liste_origin_vol) == "D":
#                     x['vol_a_transmettre'] = "ADET"
#                     x["PLN_valide"] = False
#                     x['invalidite_TU'].append("TRANSD")
#                 elif trouver_code(x["depfinal"], liste_origin_vol) == "E":
#                     if x["SVR"] == 1:
#                         x['vol_a_transmettre'] = False
#                     else:
#                         x['vol_a_transmettre'] = "ADET"
#                         x["PLN_valide"] = False
#                         x['invalidite_TU'].append("TRANSE")
#                 else:
#                     x['vol_a_transmettre'] = "ADET"
#                     x["PLN_valide"] = False
#                     x['invalidite_TU'].append("TRANSI")
#             else:
#                 if x["AVR"] == 1:
#                     x['vol_a_transmettre'] = True
#                 else:
#                     if x["balisefinal"] == "PTGEO":
#                         x['vol_a_transmettre'] = "ADET"
#                         x["PLN_valide"] = False
#                         x['invalidite_TU'].append("TRANSP")
#                     else:
#                         x['vol_a_transmettre'] = False
#     df_utile = df_utile.apply(TU_3_element, axis=1)
#     return df_utile

In [30]:
# def TU_3(df_utile):
#     def TU_3_element(x):
#         dep_value = None
#         balise_value = None

#         # Vérification des colonnes de départ
#         for col in ["deprealise", "depfinal", "depprevu"]:
#             if col in x and not pd.isna(x[col]) and x[col] != "":
#                 dep_value = x[col]
#                 break
#         for col in ["baliserealise", "balisefinal", "baliseprevu"]:
#             if col in x and not pd.isna(x[col]) and x[col] != "":
#                 balise_value = x[col]
#                 break

#         if dep_value == "ZZZZ":
#             x['PLN_valide'] = False
#             x['PLN_à_verifier_TU'] = False
#             x['invalidite_TU'].extend(["DEPAR2", "TRANS1"])
#         elif dep_value:
#             dep_code = trouver_code(dep_value, liste_origin_vol)
#             if dep_code == "A":
#                 x['vol_a_transmettre'] = False
#             elif dep_code == "B":
#                 x['vol_a_transmettre'] = True
#             elif dep_code == "C":
#                 if x["AVR"] == 1:
#                     x['vol_a_transmettre'] = "ADET"
#                     x["PLN_valide"] = False
#                     x['invalidite_TU'].append("TRANSC")
#                 else:
#                     x['vol_a_transmettre'] = False
#             elif dep_code == "D":
#                 x['vol_a_transmettre'] = "ADET"
#                 x["PLN_valide"] = False
#                 x['invalidite_TU'].append("TRANSD")
#             elif dep_code == "E":
#                 if x["SVR"] == 1:
#                     x['vol_a_transmettre'] = False
#                 else:
#                     x['vol_a_transmettre'] = "ADET"
#                     x["PLN_valide"] = False
#                     x['invalidite_TU'].append("TRANSE")
#             else:
#                 x['vol_a_transmettre'] = "ADET"
#                 x["PLN_valide"] = False
#                 x['invalidite_TU'].append("TRANSI")
#         else:
#             if x["AVR"] == 1:
#                 x['vol_a_transmettre'] = True
#             else:
#                 if balise_value == "PTGEO":
#                     x['vol_a_transmettre'] = "ADET"
#                     x["PLN_valide"] = False
#                     x['invalidite_TU'].append("TRANSP")
#                 else:
#                     x['vol_a_transmettre'] = False

#         return x

#     df_utile = df_utile.apply(TU_3_element, axis=1)
#     return df_utile

##  Algo 4

In [31]:
def TU_4(df_utile):
    def TU_4_element(x):
        x['vol_a_transmettre'] = True
        for col in ["dep_realise", "dep_final", "dep_prevu"]:
            if col in x and not pd.isna(x[col]) and x[col] != "":
                dep_value = x[col]
                break
        for col in ["arr_realise", "arr_final", "arr_prevu"]:
            if col in x and not pd.isna(x[col]) and x[col] != "":
                arr_value = x[col]
                break
        for col in ["plnActive_realise", "plnActive_final", "plnActive_prevu"]:
            if col in x and not pd.isna(x[col]) and x[col] != "":
                plnActive = x[col]
                break

        if dep_value in AERODROMES_D_APPROCHE["Code terrain"].to_list():
            x["vol_approche"] = True

        if arr_value[:2] == "LF":
            x["vol_interieur"] = True

        elif  plnActive == 0 and dep_value in AERODROMES_FRONTALIERS["Code terrain"].to_list():
            x["vol_frontalier"] = True

        return x
    df_utile = df_utile.apply(TU_4_element, axis=1)
    return df_utile

In [32]:
# def TU_4(df_utile):
#     def TU_4_element(x):
#         x['vol_a_transmettre'] = True
#         if x["depfinal"] in AERODROMES_D_APPROCHE:
#             x["vol_approche"] = True
#         if x["arrfinal"][:2] == "LF":
#             x["vol_interieur"] == True
#         elif x["PLN_activefinal"] != 1 & x["depfinal"] in AERODROMES_FRONTALIERS:
#             x["vol_frontalier"] == True
#     df_utile = df_utile.apply(TU_4_element, axis=1)
#     return df_utile

## Algo 5

In [33]:
def TU_5(df_utile):
    def TU_5_element(x):
        # Vérification des colonnes balise et regleVol
        balise_value = None
        regleVol_value = None
        dep_value = None
        for col in ["baliserealise", "balisefinal", "baliseprevu"]:
            if col in x and not pd.isna(x[col]) and x[col] != "":
                balise_value = x[col]
                break
        for col in ["regleVol_realise", "regleVol_final", "regleVol_prevu"]:
            if col in x and not pd.isna(x[col]) and x[col] != "":
                regleVol_value = x[col]
                break
        for col in ["dep_realise", "dep_final", "dep_prevu"]:
            if col in x and not pd.isna(x[col]) and x[col] != "":
                dep_value = x[col]
                break
        # Conditions basées sur les valeurs trouvées
        if x["typePln"] == "VFR" and balise_value.startswith('VFR') and regleVol_value == "V":
            x['vol_a_transmettre'] = False
        elif x["typePln"] in ["APL", "FPL", "FIH", "FII"] and balise_value.startswith('VFR'):
            x['vol_a_transmettre'] = False
            x['vol_VFR'] = True
            x["typePln"] = "VFR"
        else:
            if not x.get('vol_VFR', False) and regleVol_value == "V" and x.get('vol_a_transmettre', True):
                x["PLN_valide"] = False
                x['invalidite_TU'].append("TRANS0")
            elif dep_value and dep_value[:2] != "LF" and not x.get('vol_a_transmettre', True) and regleVol_value == "Z":
                x["PLN_valide"] = False
                x['invalidite_TU'].append("TRANS2")
                x["typePln"] = "AFI"
            if not x.get('vol_VFR', False) and any(s.startswith('TRANS') for s in x['invalidite_TU']):
                x['vol_a_transmettre'] = "ADET"
        if x["typePln"] in ["APL", "FPL"] and regleVol_value in ["Y", "Z"] and x.get('vol_a_transmettre', True):
            x["code_d_exoneration"] = "Y"
        return x
    df_utile = df_utile.apply(TU_5_element, axis=1)
    return df_utile


In [34]:
# def TU_5(df_utile):
#     def TU_5_element(x):
#         if x["typePln"] == "VFR" and x["baliseprevu"].str.startswith('VFR') and x["regleVol_prevu"] == "V":
#             x['vol_a_transmettre'] = False
#         elif (x["typePln"] == "APL" or x["typePln"] == "FPL" or x["typePln"] == "FIH" or x["typePln"] == "FII") and x["baliseprevu"].str.startswith('VFR'):
#             x['vol_a_transmettre'] = False
#             x['vol_VFR'] = True
#             x["typePln"] = "VFR"
#         else:
#             if x['vol_VFR'] == False and x["regleVol_prevu"] == "V" and x['vol_a_transmettre'] == True:
#                 x["PLN_valide"] == False
#                 x['invalidite_TU'].append("TRANS0")
#             elif x["depprevu"][:2] != "LF" and x['vol_a_transmettre'] == False and x["regleVol_prevu"] == "Z":
#                 x["PLN_valide"] == False
#                 x['invalidite_TU'].append("TRANS2")
#                 x["typePln"] = "AFI"
#             if x['vol_VFR'] == False and any(s.startswith('TRANS') for s in x['invalidite_TU']):
#                 x['vol_a_transmettre'] = "ADET"
#         if (x["typePln"] == "APL" or x["typePln"] == "FPL") and ( x["regleVol_prevu"] == "Y" or  x["regleVol_prevu"] == "Z") and x['vol_a_transmettre'] == True:
#             x["code_d_exoneration"] = "Y"
#     df_utile = df_utile.apply(TU_5_element, axis=1)
#     return df_utile

## Algo 6

In [35]:
def TU_65_element(x):
    # Fonction pour trouver une valeur valide dans les colonnes multiples
    def get_valid_value(element, columns):
        for col in columns:
            if col in element and not pd.isna(element[col]) and element[col] != "":
                return element[col]
        return None
    type_avion_value = get_valid_value(x, ["typeAvion_realise", "typeAvion_final", "typeAvion_prevu"])
    if type_avion_value in AERONEFS_STRICTEMENT_MILITAIRES['Type avion'].tolist():
        x['type_d_avion_militaire'] = True
        x['PLN_valide'] = False
        x['invalidite_TU'].extend(["TYPA19", "INDI19"])
        filtre_avion = AERONEFS_STRICTEMENT_MILITAIRES[AERONEFS_STRICTEMENT_MILITAIRES["Type avion"] == type_avion_value]
        if not pd.isna(filtre_avion["Code exoneration"].iloc[0]):
            x["code_d_exoneration"] = filtre_avion["Code exoneration"].iloc[0]
            x['code_exploitant'] = filtre_avion["Code exploitant"].iloc[0]
        else:
            x['code_exploitant'] = 'Z'
    return x

In [36]:
def TU_64_element(x):
    # Fonction pour trouver une valeur valide dans les colonnes multiples
    def get_valid_value(element, columns):
        for col in columns:
            if col in element and not pd.isna(element[col]) and element[col] != "":
                return element[col]
        return None
    type_avion_value = get_valid_value(x, ["typeAvion_realise", "typeAvion_final", "typeAvion_prevu"])
    OPR = None
    if not(pd.isna(x["case18"])):
        OPR = trouver_case18("OPR/", x["case18"])
    if not(pd.isna(x["case18"])) and OPR != "NULL" and OPR in OPERATEURS_MILITAIRES['Nom operateur'].tolist():
        operateur_militaire = OPERATEURS_MILITAIRES[OPERATEURS_MILITAIRES["Nom operateur"] == OPR]
        x["code_d_exoneration"] = operateur_militaire["Code exoneration"].iloc[0]
        x['code_exploitant'] = operateur_militaire["Code exploitant"].iloc[0]
    else:
        if type_avion_value in AERONEFS_STRICTEMENT_MILITAIRES['Type avion'].tolist():
            x['type_d_avion_militaire'] = True
            avion_militaire = AERONEFS_STRICTEMENT_MILITAIRES[AERONEFS_STRICTEMENT_MILITAIRES["Type avion"] == type_avion_value]
            if not pd.isna(avion_militaire["Code exoneration"].iloc[0]):
                x["code_d_exoneration"] = avion_militaire["Code exoneration"].iloc[0]
                x['code_exploitant'] = avion_militaire["Code exploitant"].iloc[0]
            else:
                x['code_exploitant'] = 'Z'
        x['PLN_valide'] = False
        x['invalidite_TU'].extend(["EXO19", "INDI19", "OPR19"])
    return x

In [37]:
def TU_63_element(x):
    # Fonction pour trouver une valeur valide dans les colonnes multiples
    def get_valid_value(element, columns):
        for col in columns:
            if col in element and not pd.isna(element[col]) and element[col] != "":
                return element[col]
        return None
    call_sign_value = get_valid_value(x, ["callSign_realise", "callSign_final", "callSign_prevu"])
    if call_sign_value and trouver_pattern(call_sign_value, IMMATRICULATION_CORRECTE["Immatriculation"]):
        if trouver_pattern(call_sign_value, IMMATRICULATION_A_VERIFIER["Immatriculation"]):
            x['PLN_valide'] = False
            x['invalidite_TU'].extend(["IVDIC$", "EXO$", "OPR$"])
        else:
            x['type_d_indicatif'] = "IM"
        x = TU_65_element(x)
    else:
        x = TU_64_element(x)
    return x

In [38]:
def TU_62_element(x):
    def get_valid_value(element, columns):
        for col in columns:
            if col in element and not pd.isna(element[col]) and element[col] != "":
                return element[col]
        return None
    call_sign_value = get_valid_value(x, ["callSign_realise", "callSign_final", "callSign_prevu"])
    if call_sign_value and call_sign_value[:2] in COMPAGNIES_BIGRAMMES_ET_SUFFIXES["Indicatif"].values:
        x['code_exploitant'] = COMPAGNIES_BIGRAMMES_ET_SUFFIXES[COMPAGNIES_BIGRAMMES_ET_SUFFIXES["Indicatif"] == call_sign_value[:2]]["Code exploitant"].iloc[0]
        if x['vol_interieur'] == True:
            x["code_d_exoneration"] = COMPAGNIES_BIGRAMMES_ET_SUFFIXES[COMPAGNIES_BIGRAMMES_ET_SUFFIXES["Indicatif"] == call_sign_value[:2]]["Code exoneration"].iloc[0]
            x['compagnie_française'] = True
            x['type_d_indicatif'] = "BI"
        else:
            x['PLN_valide'] = False
            x['invalidite_TU'].extend(["INDIC%", "EXO%", "OPR%"])
        x = TU_65_element(x)
    return x

In [39]:
def TU_612_element(x):
    def get_valid_value(element, columns):
        for col in columns:
            if col in element and not pd.isna(element[col]) and element[col] != "":
                return element[col]
        return None

    call_sign_value = get_valid_value(x, ["callSign_realise", "callSign_final", "callSign_prevu"])

    if call_sign_value and re.search(r'\d[A-Z]$', call_sign_value) and call_sign_value[-1] == -1:
        x['PLN_valide'] = False
        x['invalidite_TU'].append("INDIC6")
    x = TU_65_element(x)
    return x

In [40]:
def TU_611_element(x):
    def get_valid_value(element, columns):
        for col in columns:
            if col in element and not pd.isna(element[col]) and element[col] != "":
                return element[col]
        return None
    call_sign_value = get_valid_value(x, ["callSign_realise", "callSign_final", "callSign_prevu"])
    if call_sign_value and call_sign_value[-1] in LETTRES_AIR_FRANCE['Lettre'].tolist():
        code_air_france = LETTRES_AIR_FRANCE[LETTRES_AIR_FRANCE['Lettre'] == call_sign_value[-1]]["Code exo/trait"].iloc[0]
        if code_air_france == 2:
            x['PLN_valide'] = False
            x['invalidite_TU'].extend(["INDIC8", "EXO8"])
        elif not code_air_france.isdigit():
            x["code_d_exoneration"] = code_air_france
    else:
        x['PLN_valide'] = False
        x['invalidite_TU'].append("EXO10")
    x = TU_65_element(x)
    return x

In [41]:
def TU_61_element(x):
    def get_valid_value(element, columns):
        for col in columns:
            if col in element and not pd.isna(element[col]) and element[col] != "":
                return element[col]
        return None
    call_sign_value = get_valid_value(x, ["callSign_realise", "callSign_final", "callSign_prevu"])
    if call_sign_value[:3] in COMPAGNIES_AVEC_TRIGRAMME["Trigramme"].tolist():
        x['type_d_indicatif'] = "TR"
        if trouver_pattern(call_sign_value, INDICATIFS_TRIGRAMME_A_VERIFIER["Indicatif"]):
            x['PLN_valide'] = False
            x['invalidite_TU'].extend(["IVDIC5", "EXO5"])
        elif call_sign_value[:3] == "AFR":
            x['compagnie_française'] = True
            if re.search(r'\d[A-Z]$', call_sign_value):
                x = TU_611_element(x)
            else:
                x = TU_65_element(x)
        else:
            x = TU_612_element(x)
    else:
        x = TU_64_element(x)
    return x

In [42]:
def TU_6(df_utile):
    def TU_6_element(x):
        def get_valid_value(element, columns):
            for col in columns:
                if col in element and not pd.isna(element[col]) and element[col] != "":
                    return element[col]
            return None
        call_sign_value = get_valid_value(x, ["callSign_realise", "callSign_final", "callSign_prevu"])
        if call_sign_value and trouver_pattern(call_sign_value, INDICATIF_A_CODE_AUTO["Indicatif"].tolist()):
            Indicatif = renvoie_pattern(call_sign_value, INDICATIF_A_CODE_AUTO["Indicatif"].tolist())
            x["code_d_exoneration"] = INDICATIF_A_CODE_AUTO[INDICATIF_A_CODE_AUTO["Indicatif"] == Indicatif]["Code exoneration"].iloc[0]
            x['code_exploitant'] = INDICATIF_A_CODE_AUTO[INDICATIF_A_CODE_AUTO["Indicatif"] == Indicatif]["Code exploitant"].iloc[0]
            if x["code_d_exoneration"] in ["X", "M"]:
                x['type_d_avion_militaire'] = True
        else:
            if trouver_pattern(call_sign_value, INDICATIFS_A_STRUCTURE_TRIGRAMME_CORRECTE["Indicatif"].tolist()):
                x = TU_61_element(x)
            elif re.match(r'^[A-Z]{2}\d{3}[A-Z]{2}$', call_sign_value) and call_sign_value[:2] in COMPAGNIES_BIGRAMMES_ET_SUFFIXES["Bigramme"].tolist():
                x = TU_62_element(x)
            else:
                x = TU_63_element(x)
        return x
    df_utile = df_utile.apply(TU_6_element, axis=1)
    return df_utile

## Algo 7

In [43]:
# Chaîne d'entrée
def trouver_case18(prefix, input_string):
    index_find = input_string.find(prefix)
    if index_find == -1:
        return "NULL"
    else:
        start_index = index_find + len(prefix)
        end_index = input_string.find(' ', start_index)
        opr_code = input_string[start_index:end_index]
        return opr_code

In [44]:
# def TU_7(df_utile):
#     def TU_7_element(x):
#         if x["PLN_activeprevu"] == "0" and x['vol_a_transmettre'] == True:
#             x['PLN_valide'] = False
#         if x["callSign_prevu"] == "APL" or x["callSign_prevu"] == "FPL":
#             if x['vol_a_transmettre'] == True:
#                 if x["code_d_exoneration"] == "Y" or ["code_d_exoneration"] == "Z":
#                     if x['typeVol_prevu'] == "M":
#                         x['PLN_valide'] = False
#                         x['invalidite_TU'].append("EXO10")
#                         x['invalidite_TU'].append("OPR10")
#                     else:
#                         if trouver_case18("RMK/", x["case18"]) == "TRAINING" and x['vol_interieur'] == True:
#                             x["code_d_exoneration"] = "T"
#                         elif x['typeVol_prevu'] == "X" and trouver_case18("RMK/", x["case18"]) != "EVASAN":
#                             x['PLN_valide'] = trouver_case18
#                             x['invalidite_TU'].append("EXO12")
#             else:
#                 if trouver_case18("RMK/", x["case18"]) == "TRAINING" and x['vol_interieur'] == True and (x["code_d_exoneration"] == "Y" or ["code_d_exoneration"] == "Z"):
#                     x["code_d_exoneration"] = "T"

#     df_utile = df_utile.apply(TU_7_element, axis=1)
#     return df_utile

In [45]:
def TU_7(df_utile):
    def TU_7_element(x):
        # Fonction pour trouver une valeur valide dans les colonnes multiples
        def get_valid_value(element, columns):
            for col in columns:
                if col in element and not(element[[col]].isna().iloc[0]):
                    return element[col]
            return None

        PLN_active_value = get_valid_value(x, ["plnActive_realise", "plnActive_final", "plnActive_prevu"])
        call_sign_value = get_valid_value(x, ["callSign_realise", "callSign_final", "callSign_prevu"])
        type_vol_value = get_valid_value(x, ["typeVol_realise", "typeVol_final", "typeVol_prevu"])

        if PLN_active_value == "0" and x.get('vol_a_transmettre', False):
            x['PLN_valide'] = False

        if call_sign_value in ["APL", "FPL"]:
            if x.get('vol_a_transmettre', False):
                if code_exoneration_value in ["Y", "Z"]:
                    if type_vol_value == "M":
                        x['PLN_valide'] = False
                        x['invalidite_TU'].extend(["EXO10", "OPR10"])
                    else:
                        if trouver_case18("RMK/", x["case18"]) == "TRAINING" and x.get('vol_interieur', False):
                            x["code_d_exoneration"] = "T"
                        elif type_vol_value == "X" and trouver_case18("RMK/", x["case18"]) != "EVASAN":
                            x['PLN_valide'] = False
                            x['invalidite_TU'].append("EXO12")
            else:
                if trouver_case18("RMK/", x["case18"]) == "TRAINING" and x.get('vol_interieur', False) and code_exoneration_value in ["Y", "Z"]:
                    x["code_d_exoneration"] = "T"
        return x
    df_utile = df_utile.apply(TU_7_element, axis=1)
    return df_utile

## Traitement unitaire complet

In [46]:
def traitement_unitaire(df_utile):
    df_utile = TU_init(df_utile)
    df_utile = TU_1(df_utile)
    df_utile = TU_2(df_utile)

    def check_depfinal_prefix(row):
        for col in ["dep_realise", "dep_final", "dep_prevu"]:
            if col in row and not pd.isna(row[col]) and row[col] != "":
                if row[col][:2] == "LF":
                    return True
        return False

    def apply_transformations(row):
        if check_depfinal_prefix(row):
            return TU_4(pd.DataFrame([row])).iloc[0]
        else:
            return TU_3_bis(pd.DataFrame([row])).iloc[0]

    # Appliquer les transformations par ligne
    df_utile = df_utile.apply(apply_transformations, axis=1)

    df_utile = TU_5(df_utile)
    df_utile = TU_6(df_utile)
    df_utile = TU_7(df_utile)
    return df_utile

# Transformation des données RDVC

In [47]:
jour_1_utile_inutile = traitement_utile_inutile(jour_1)
jour_2_utile_inutile = traitement_utile_inutile(jour_2)
jour_3_utile_inutile = traitement_utile_inutile(jour_3)

In [48]:
jour_1_utile = jour_1_utile_inutile[jour_1_utile_inutile["utile_inutile"] == "UTI"].copy()
jour_2_utile = jour_2_utile_inutile[jour_2_utile_inutile["utile_inutile"] == "UTI"].copy()
jour_3_utile = jour_3_utile_inutile[jour_3_utile_inutile["utile_inutile"] == "UTI"].copy()

In [49]:
jour_1_utile_traitee = traitement_unitaire(jour_1_utile)
jour_2_utile_traitee = traitement_unitaire(jour_2_utile)
jour_3_utile_traitee = traitement_unitaire(jour_3_utile)

Fonction
ajouter pour chaque df une colonne doublon à False
Verifier que si un call sign et une heure de reference à 19mn pret (à parametrer) indiquer doublon sur l'un des vols seulement

In [50]:
seuil_doublon = 19

In [51]:
import pandas as pd

def ajouter_colonne_doublon(df, tolerance_minutes=19):
    df['doublon'] = False

    df = df.sort_values(by=['callSign_prevu', 'heure_de_reference']).reset_index(drop=True)
    i = 1
    while i < len(df):
        if df.at[i, 'callSign_prevu'] == df.at[i - 1, 'callSign_prevu']:
            delta = abs((df.at[i, 'heure_de_reference'] - df.at[i - 1, 'heure_de_reference']))
            if delta <= tolerance_minutes:
                df = df.drop(i).reset_index(drop=True)
                continue  # Réévaluer le même index i après la suppression et la réinitialisation de l'index
        i += 1
    return df

def supprimer_doublons(df):
    return df[df['doublon'] == False].drop(columns=['doublon'])

In [52]:
jour_1_utile_traitee_avec_doublons_false = ajouter_colonne_doublon(jour_1_utile_traitee)
jour_2_utile_traitee_avec_doublons_false = ajouter_colonne_doublon(jour_2_utile_traitee)
jour_3_utile_traitee_avec_doublons_false = ajouter_colonne_doublon(jour_3_utile_traitee)

jour_1_utile_traitee_sans_doublons = supprimer_doublons(jour_1_utile_traitee_avec_doublons_false)
jour_2_utile_traitee_sans_doublons = supprimer_doublons(jour_2_utile_traitee_avec_doublons_false)
jour_3_utile_traitee_sans_doublons = supprimer_doublons(jour_3_utile_traitee_avec_doublons_false)

<ipython-input-51-9a6372dd0a08>:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['doublon'] = False
<ipython-input-51-9a6372dd0a08>:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['doublon'] = False
<ipython-input-51-9a6372dd0a08>:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['doublon'] 

In [131]:
def get_valid_value(element, columns):
        for col in columns:
            if col in element and not(element[[col]].isna().iloc[0]):
                return element[col]
        return ""

def format_flight_message(row):
    sequence_number = f"{row.name + 1:04}"
    intro_correction_code = "F"
    valeur = int(get_valid_value(row, ['heure_de_reference']))
    heure = (valeur%1440)//60
    min = (valeur%1440)%60
    time_of_departure = f"{(100*heure + min):04}"
    aerodrome_of_departure = get_valid_value(row, ['dep_realise', 'dep_final', 'de_prevu']).ljust(4)
    aerodrome_of_arrival = get_valid_value(row, ['arr_realise', 'arr_final', 'arr_prevu']).ljust(4)
    flight_identification = get_valid_value(row, ['callSign_realise', 'callSign_final', 'callSign_prevu']).ljust(9)
    main_exemption_code = row["code_d_exoneration"].ljust(1)
    aircraft_type = get_valid_value(row, ['typeAvion_realise', 'typeAvion_final', 'typeAvion_prevu']).ljust(7)
    operator = str(row["code_exploitant"]).ljust(3)
    aircraft_registration = str(get_valid_value(row, ['work1realise', 'work1final', 'work1prevu'])).ljust(9)
    comment = ""
    est_off_block_date = get_valid_value(row, ['date_de_reference']).ljust(6)
    ifplid = get_valid_value(row, ['IFPL_realise', 'IFPL_final', 'IFPL_prevu']).ljust(9)
    initial_aerodrome_destination = ""
    charging_zone_overflown = ""
    entry_point_coordinates = ""
    exit_point_coordinates = ""
    supplementary_exemption_codes = ""
    source_icao_address = ""
    icao_address = get_valid_value(row, ['adresseModeS_realise', 'adresseModeS_final', 'adresseModeS_prevu']).ljust(6)
    additional_comment = ""
    front_alg_fr = row["front_alg_fr"]
    premier_plot_fr = str(row["premier_plot_fr"])
    modes_fr = str(row["modes_fr"])

    return {
        "Sequence number": sequence_number,
        "Code": intro_correction_code,
        "Time of departure/entry": time_of_departure,
        "Departure aerodrome": aerodrome_of_departure,
        "Arrival aerodrome": aerodrome_of_arrival,
        "Flight identification": flight_identification,
        "Main Exemption code": main_exemption_code,
        "Type of aircraft": aircraft_type,
        "Operator": operator,
        "Aircraft Registration": aircraft_registration,
        "Comment1": comment,
        "Flight date": est_off_block_date,
        "IFPLID": ifplid,
        "Planned_aerodrome": initial_aerodrome_destination,
        "Charging_zone_overflow": charging_zone_overflown,
        "Entry_point": entry_point_coordinates,
        "Exit_point": exit_point_coordinates,
        "Sup_exemption_code": supplementary_exemption_codes,
        "Source of the Aircraft Address": source_icao_address,
        "24-bit Aircraft Address": icao_address,
        "Comment2": additional_comment,
        "case7" : str(row["case7"]),
        "case8" : str(row["case8"]),
        "case9" : str(row["case9"]),
        "case10" : str(row["case10"]),
        "case13" : str(row["case13"]),
        "case15" : str(row["case15"]),
        "case16" : str(row["case16"]),
        "case18" : str(row["case18"]),
        "ccrArrival" : str(row["ccrArrival"]),
        "front_alg_fr" : front_alg_fr,
        "premier_plot_fr" : premier_plot_fr,
        "modes_fr" : modes_fr
    }


In [54]:
jour_123 = pd.concat([jour_1_utile_traitee_avec_doublons_false, jour_2_utile_traitee_avec_doublons_false, jour_3_utile_traitee_avec_doublons_false], ignore_index=True)
jour_123['date_de_reference'] = jour_123['date_de_reference'].dt.strftime('%y%m%d')
jour_123 = jour_123[jour_123["vol_a_transmettre"] == True]

In [55]:
data = jour_123.copy()

In [144]:
jour_123 = data.copy()

In [145]:
nom_fichier_survol = f'data_survols_{datetime.strptime(dateAnalyse, "%d-%m-%Y").strftime("%Y%m%d")}.csv'
data_survols = pd.read_csv(nom_fichier_survol, sep=',')
def conversion_dec_hexa(number):
    if number != number:  # Vérification pour NaN
        return 'NaN'
    else:
        hex_value = hex(int(number))[2:].upper()
        return hex_value.zfill(6)
data_survols['MODES_ADRESS'] = data_survols['MODES_ADRESS'].apply(conversion_dec_hexa)
# Créer les colonnes front_alg_fr, premier_plot_fr, modes_fr avec des NaN
jour_123['front_alg_fr'] = np.nan
jour_123['premier_plot_fr'] = np.nan
jour_123['modes_fr'] = np.nan

# Parcourir chaque ligne de jour_123
for i, row in jour_123.iterrows():
    # Filtrer data_survols pour trouver les lignes où les critères sont respectés
    matched_rows = data_survols[
        (row['callSign_prevu'] == data_survols['FLPL_CALL_SIGN'])
        # (row['dep_prevu'] == data_survols['FLPL_DEPR_AIRP']) &
        # (row['arr_prevu'] == data_survols['FLPL_ARRV_AIRP'])
    ]

    if not matched_rows.empty:
        # Si une correspondance est trouvée, prendre la première (ou vous pouvez choisir une autre règle)
        match = matched_rows.iloc[0]

        # Mettre à jour les colonnes de jour_123
        jour_123.at[i, 'front_alg_fr'] = match['frontiere_algerienne']
        jour_123.at[i, 'premier_plot_fr'] = match['premier_plot']
        jour_123.at[i, 'modes_fr'] = match['MODES_ADRESS']


<ipython-input-145-37befb3a656d>:29: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'False' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  jour_123.at[i, 'front_alg_fr'] = match['frontiere_algerienne']
<ipython-input-145-37befb3a656d>:31: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '4D4038' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  jour_123.at[i, 'modes_fr'] = match['MODES_ADRESS']


In [146]:
jour_123 = jour_123.apply(format_flight_message, axis=1).tolist()
jour_123 = pd.DataFrame(jour_123)
jour_123['Time of departure/entry'] = pd.to_numeric(jour_123['Time of departure/entry'], errors='coerce')

In [147]:
modeS1 = pd.read_csv(os.path.join(chemin_dossier_table_systeme,"modeS1"), sep=';', skiprows=3)
modeS1 = modeS1[['Mode S', 'Type Avion']]
modeS1.rename(columns={
    'Type Avion': 'Type Avion Survol'
}, inplace=True)


jour_123['Type_Avion_Survol'] = np.nan
# Parcourir chaque ligne de jour_123
for i, row in jour_123.iterrows():
    # Filtrer data_survols pour trouver les lignes où les critères sont respectés
    matched_rows = modeS1[
        (row['modes_fr'] == modeS1['Mode S'])
    ]

    if not matched_rows.empty:
        # Si une correspondance est trouvée, prendre la première (ou vous pouvez choisir une autre règle)
        match = matched_rows.iloc[0]
        # Mettre à jour les colonnes de jour_123
        jour_123.at[i, 'Type_Avion_Survol'] = match['Type Avion Survol']


<ipython-input-147-379ce0c7e39c>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  modeS1.rename(columns={
<ipython-input-147-379ce0c7e39c>:20: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'C525' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  jour_123.at[i, 'Type_Avion_Survol'] = match['Type Avion Survol']


In [149]:
import pandas as pd
from google.colab import files

# Sauvegarder le DataFrame en fichier CSV
jour_123.to_csv('jou_123_survol.csv', index=False)

# Télécharger le fichier CSV sur votre ordinateur local
files.download('jou_123_survol.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>